# Getting data using AFDC API

In [ ]:
import requests
import csv
import pandas as pd
import json
import os
import geopandas as gpd
import geodatasets
import matplotlib.pyplot as plt
import numpy as np
import re
import datetime
import seaborn as sns




In [ ]:
#with open('api_keys.json') as f:
#    keys = json.load(f)
#key = keys['keys']['afdc']

In [ ]:
# acquire API key from this link: https://developer.nrel.gov/docs/
# https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/
# note to download as json, then convert to csv in next cell


key = 'TODO'
api_key = key
url_all = f'https://developer.nrel.gov/api/alt-fuel-stations/v1.json?api_key={api_key}'
url_ev_CA = f'https://developer.nrel.gov/api/alt-fuel-stations/v1.json?api_key={api_key}&fuel_type=ELEC&state=CA&'
url_geo_allfuels = f'https://developer.nrel.gov/api/alt-fuel-stations/v1.geojson?api_key={api_key}&state=CA'

# set the filepath to save these datasets to. These datasets WILL be used in other notebooks. 
fp = 'TODO'
fp = 'datasets/ev_stations/'

In [ ]:
datasets = {
    'fullset':url_all,
    'ev_CA':url_ev_CA,
}

def update_datasets(urldict, fp = 'data/'):
    for key in urldict.keys():

        url = urldict[key]
        filename = key

        #just to update and skip the ones we have already
        path = f'{fp}{filename}.csv' 
        if os.path.exists(path):
            continue

        with requests.get(url) as response:
            data = response.json()
        print(f"dataset {filename} collected")
        
        #uncomment if want to keep json files too (?), as this only keeps the fuelstations
        #with open(f"{filename}.json", "w") as outfile:
        #    json.dump(data, outfile)

        df = pd.DataFrame(data['fuel_stations'])
        df.to_csv(f'{fp}{filename}.csv')
        print(f"dataset {fp}{filename} saved to csv")


In [ ]:
update_datasets(datasets,fp)

Loading dataset into dataframe

In [ ]:
df_full = pd.read_csv(f'{fp}fullset.csv')
df_ev_CA = pd.read_csv(f'{fp}ev_CA.csv')

In [ ]:
df_full.shape, df_ev_CA.shape

In [ ]:
#  source: https://www.pepma-ca.com/public/OtherResources.aspx?f=9KwtCRtf%2BqXCw1HmKSHHUjkyQ0VzK1rPzZozetSiP8jtWC2ehoHI6DXFZTRFfVEca3P%2B1ddq%2BkM%3D
sdge_zips = pd.read_excel('https://www.pepma-ca.com/public/OtherResources.aspx?f=9KwtCRtf%2BqXCw1HmKSHHUjkyQ0VzK1rPzZozetSiP8jtWC2ehoHI6DXFZTRFfVEca3P%2B1ddq%2BkM%3D')
sdge_zips = sdge_zips.ZIP_CODE

In [ ]:
temp = df_ev_CA[df_ev_CA.zip.isin(sdge_zips)]
temp = temp[temp.fuel_type_code =="ELEC"]
ev_chargers_in_SDGE = temp.shape[0]
ev_chargers_in_SDGE

### Wrangling the data

In [ ]:
df_full['street_address'].nunique()
df_full.columns

#columns we want to keep/use in this nb
cols_keep = [
    'fuel_type_code',
    'status_code',
    'open_date',
    'expected_date',
    'station_name',
    'street_address',
    'state',
    'city',
    'zip',
    'latitude',
    'longitude',
    'id',
    'ev_level1_evse_num',
    'ev_level2_evse_num',
    'ev_dc_fast_num',
    'ev_other_evse'

]
def trim_cols(df):
    return df[cols_keep]

df_full = trim_cols(df_full)

df_full.head()

In [ ]:
#finding missing values
print(df_full.isna().sum())

#notice 224 open dates missing; Note that most propane (LPG) stations do not have open dates.
print(len(df_full[df_full.status_code == 'P']))

In [ ]:
#let's standardize the strings for street address. 
df_full['street_address'] = df_full['street_address'].str.lower()
df_full.head()

In [ ]:
#let's see if there are duplicates, and if so what's going on.
print('num of street address dupes', len(df_full) - df_full.street_address.nunique())
#seems there are quite a few duplicated street addresses - what about station names/ids? This will help us see if they're just close to each other or actually dupes.
print('num id dupes',len(df_full) - df_full.id.nunique()) #seems that all the ids are unique.
print('num station name dupes', len(df_full) - df_full.station_name.nunique()) #seems that there are actually some duplicate station names. Let's see what's going on?
df_full.station_name.value_counts() #seems like there is a generic naming schema for some chargers. Let's see if these same-named chargers exist in the same location!
dupes = df_full.groupby('station_name').street_address.value_counts().sort_values(ascending=False)
dupes[dupes>1] #seems like there might be some actual dupes. would be interesting to look into. for now, let's take a look at the most common one: petro ontario at 4325 e guasti rd.

In [ ]:
df_full[df_full.station_name == 'Petro Ontario']
#seems like they all opened on the same day, but one uses Renewable Diesel and the rest use Bio Diesel
#for now, leave it here, might be worth investigating ways to check for actual duplicates or identify them in other ways. For now, since they all have unique IDs, they will count as individual stations.

### Calculating num of SDGE ev chargers

In [ ]:

sdge_ret = df_ev_CA[df_ev_CA.zip.isin(sdge_zips)]
len(sdge_ret)

## Let's see how many of each station each state has

In [ ]:
fuels_by_state = df_full.groupby(['state','fuel_type_code']).size()
fuels = fuels_by_state.reset_index()
fuels

In [ ]:

fig,ax = plt.subplots(figsize = (25,10))

sns.histplot(
    ax = ax,
    data = df_full,
    x = 'state',
    hue = 'fuel_type_code',
    #kind='bar'
    multiple='dodge',
)


In [ ]:
fig,ax = plt.subplots(nrows = 7, ncols = 1, figsize = (40,40))

for fuel, ax in list(zip(fuels.fuel_type_code.unique(), ax)):
    
    data = fuels[fuels['fuel_type_code'] == fuel]
    ax.set_title(fuel)
    sns.barplot(
        ax = ax,
        data = data,
        x = 'state',
        y = 0,
        order=data.sort_values(ascending = False, by= 0).state
    )

## Plotting ev charging stations

In [ ]:
#getting geojson data 

with requests.get(url_geo_allfuels) as response:
    geodata = response.json()


In [ ]:
url_usa = 'https://raw.githubusercontent.com/PublicaMundi/MappingAPI/refs/heads/master/data/geojson/us-states.json'
with requests.get(url_usa) as response:
    us_geo = response.json()


In [ ]:
#data from https://gis.data.ca.gov/datasets/CAEnergy::california-electric-transmission-lines-1/about
transm_lines = gpd.GeoDataFrame.from_file('https://stg-arcgisazurecdataprod3.az.arcgis.com/exportfiles-28775-2004/Transmission_Line_-3224603581692172535.geojson?sv=2018-03-28&sr=b&sig=OzxT2oThj2qJw5jt9xCmFG%2BV%2BIftsZojexkuZA6NfM4%3D&se=2024-12-04T03%3A50%3A20Z&sp=r')
transm_lines.head(3)

In [ ]:
#zipcode geodata from https://gis.data.ca.gov/datasets/CDEGIS::california-zip-codes/about
zip_geodata = gpd.GeoDataFrame.from_file('https://stg-arcgisazurecdataprod3.az.arcgis.com/exportfiles-39966-259/ZipCodes_-1049704744535259894.geojson?sv=2018-03-28&sr=b&sig=8vnjMFNkWmcmve84YoHGzsaI%2F2KEFu4v9KGjuXwrXWs%3D&se=2024-12-04T03%3A52%3A50Z&sp=r')
zip_geodata.head()

In [ ]:
sdge_zip_geo = zip_geodata[zip_geodata['ZIP_CODE'].astype(int).isin(sdge_zips)]
#making the geodf for sdge territory

### State abbrv to name for following eda

In [ ]:
abbrv_to_name = {
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#States.
    "AK": "Alaska",
    "AL": "Alabama",
    "AR": "Arkansas",
    "AZ": "Arizona",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "IA": "Iowa",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "MA": "Massachusetts",
    "MD": "Maryland",
    "ME": "Maine",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MO": "Missouri",
    "MS": "Mississippi",
    "MT": "Montana",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "NE": "Nebraska",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NV": "Nevada",
    "NY": "New York",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VA": "Virginia",
    "VT": "Vermont",
    "WA": "Washington",
    "WI": "Wisconsin",
    "WV": "West Virginia",
    "WY": "Wyoming",
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Federal_district.
    "DC": "District of Columbia",
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Inhabited_territories.
    "AS": "American Samoa",
    "GU": "Guam GU",
    "MP": "Northern Mariana Islands",
    "PR": "Puerto Rico PR",
    "VI": "U.S. Virgin Islands",
}

### now auto-mapping states to the names for simpler map structure

In [ ]:
#gets all states extant in geodata
l = geodata['features']
key = 'state'
states = set(x['properties'][key] for x in l)
states


geos_to_include = [abbrv_to_name[x] for x in states] #can include various features if needed
geofeats = us_geo['features']
specific_geo = [x for x in geofeats if  x['properties']['name'] in geos_to_include]
specific_geo = {'type' : 'FeatureCollection', 'features' : specific_geo}
geos_to_include



In [ ]:
USMAP = gpd.GeoDataFrame.from_features(specific_geo)
#USMAP.set_crs(crs = 'WGS84')
geodf = gpd.GeoDataFrame.from_features(geodata, crs = 4326)
#geodf.set_crs(crs = 'WGS84')
#geodf.plot(marker='*', color='green', markersize=5)
#temp1 = geodf.to_crs(USMAP.crs)

base = USMAP.plot(figsize = (20,20))
geodf.plot(ax=base, marker='o', color='r', markersize=1)



Turns out these next 2 cells are no longer needed. If there seems to be an issue with the points, uncomment and run these cells.

In [ ]:
""" 
import shapely

# seems that there are two points where coords are messed up... let's fix that. 
find = geodf[geodf['street_address'] == '500 Hotel Cir N']
row = find.index

pt = geodf[geodf['street_address'] == '500 Hotel Cir N']['geometry']
print(pt)
#seems like it's missing a negative sign in the longitude. 

x = pt.x
y = pt.y
pt = bll = shapely.Point(-x,y)
pt

geodf['geometry'][row] = pt
r = geodf[geodf['street_address'] == '500 Hotel Cir N']
r.geometry
## seems first one fixed. 
"""


In [ ]:
# now the second one!
"""
pt = geodf[geodf['street_address'] == '100 Padre Blvd']
print(pt.geometry, pt.station_name, pt.zip, pt.city, pt.state)
#seems like it's in texas. There doesn't exist a 100 Padre Blvd in ca, and the station name, zip, and city all line up with the texas location, not the CA one. We're just gonna drop this one.
geodf = geodf[geodf['street_address'] != '100 Padre Blvd']
"""


In [ ]:

base = USMAP.plot(figsize = (10,10), color='grey')
geodf.plot(ax=base, marker='o', color='y', markersize=1)

base = USMAP.plot(figsize = (10,10), color='grey')
geodf.plot(ax=base, marker='o', color='y', markersize=1)
transm_lines.plot(ax=base, linewidth = 0.5)
#with transmission lines overlaid
#much better. 


In [ ]:
# now stratify this map by fuel type. 

base = USMAP.plot(figsize = (15,15), color='lightgrey')
transm_lines.plot(ax=base, color='orange', linewidth = 0.5)
geodf.plot(column = 'fuel_type_code', ax=base, marker='o', cmap='tab10', markersize=1.5, legend = True)

#full.legend(list(fuel_dict.keys()))
#list(fuel_dict.keys())
# c=geodf['fuel_label_num'],cmap='prism',

In [ ]:
#taking a closer look at sdge territory
fig, ax = plt.subplots(figsize=(10, 10))
base = USMAP.plot(ax = ax, color='lightgrey')
transm_lines.plot(ax=ax, color='orange', linewidth = 0.5)
geodf.plot(column = 'fuel_type_code', ax=ax, marker='o', cmap='tab10', markersize=1.5, legend = True, zorder=2)
sdge_zip_geo.plot(ax = ax, color = 'lightblue',zorder=1)
ax.set_xlim(-118.5,-116)
ax.set_ylim(32.5,34)

In [ ]:
#to change state/add data just change the var url_geo to whatever params 
#unfortunately adding it to a figure removes the legend. 


#import folium


sdoh = gpd.read_file(geodatasets.get_path('geoda us_sdoh'))
trimmed_sdoh = sdoh[sdoh['state'].isin(geos_to_include)]
m = trimmed_sdoh.explore(
    figsize = (1200,800),
    #cmap='Oranges',
    column="ep_pci",  # per capita income estimate
    #scheme="naturalbreaks",  # use mapclassify's natural breaks scheme
    legend=True,  # show legend
    k=10,  # use 10 bins
    tooltip=False,  # hide tooltip
    popup=["ep_pci"],  # show popup (on-click)
    legend_kwds=dict(colorbar=False),  # do not use colorbar
    name="PCI",  # name of the layer in the map
)

#f = folium.Figure(width=1200, height=800)
geodf.explore(
    m = m,
    cmap = 'Oranges',
    column = 'fuel_type_code',
    tooltip = 'street_address',
    #marker_type = 'circle',
    marker_kwds=dict(radius=3, fill=False),
    legend = True,
    tooltip_kwds=dict(labels=False),
    width = '100%',
    height = '100%',
)
m
# this is the fuel stations overlaid on top of a chloropleth map of estimated per capita income estimate of each region. 


In [ ]:
m1 = zip_geodata.explore(
    figsize = (1200,800),
    column="POP_SQMI",  # population per square mile metric for density
    #scheme="naturalbreaks",  # use mapclassify's natural breaks scheme
    legend=True,  # show legend
    k=10,  # use 10 bins
    tooltip=False,  # hide tooltip
    popup=["POP_SQMI"],  # show popup (on-click)
    legend_kwds=dict(colorbar=False),  # do not use colorbar
    name="pop",  # name of the layer in the map
    zorder = 2
)

geodf.explore(
    m = m1,
    cmap = 'Oranges',
    column = 'fuel_type_code',
    tooltip = 'street_address',
    marker_type = 'circle',
    marker_kwds=dict(radius=50, fill=False),
    legend = True,
    tooltip_kwds=dict(labels=False),
    width = '100%',
    height = '100%',
    zorder=1
)
m1

#stations overlaid against population density

## Primary takeaways and insights from geographical data analysis:
Most notably from the above two interactive maps, it seems that while fuel stations are often NEARBY higher per capita income zones, it is less common that they actually are INSIDE the wealthiest locales; from the second map m1 above, the stations appear to be clustered around higher population density locations, as well as generally along major motorways. This makes sense, as higher population locations are usually urban areas, and the more people there are in a certain location, the more vehicle owners there will be. Additionally, from the static maps above, we can see that EV chargers follow closely along major power transmission lines. Again, this makes perfect sense, as EV chargers are by nature held in lockstep by the local energy infrastructure. To perform additional research and hypothesis testing on these assertions, I believe it would be valuable to look at data containing EV sales by zip code to see if the distributions line up with the fuel station densities we see here. As for the EV charger location in relation to power lines, I think it would be interesting to plot traditional gasoline pumps, simply to see if the power lines follow places people travel more commonly, and if the clustering and distribution is actually a function of throughput traffic rather than infrastructure design. 

## Timeseries EDA

In [ ]:

ts_df = df_full[['state', 'open_date','fuel_type_code']]
ts_df['station_added'] = np.ones(ts_df.shape[0])
ts_df

In [ ]:
ts_df.isna().sum()

In [ ]:
ts_df = ts_df.dropna()
ts = pd.Series(data = ts_df['station_added'])
ts.index = ts_df['open_date']
ts.sort_index()
# DATE input error...

In [ ]:
#fixing date error 
dates = ts_df['open_date']
dates = dates.replace('^00', '20', regex = True)
dates = pd.to_datetime(dates, format ='mixed')

#indexing vals by date
ts_df = ts_df.drop('open_date', axis = 1).set_index(dates)
ts_df = ts_df.sort_index()

ts = pd.Series(data = ts_df['station_added'])
ts = ts.sort_index()
ts


In [ ]:

ts.cumsum().plot(xlim=(min(ts.index),datetime.datetime.now()))


In [ ]:
ts_df = ts_df[['state','station_added']]
ts_df.groupby('state').sum().sort_values('station_added')

In [ ]:
ts_df[ts_df.state == 'OH'].cumsum().plot()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
for cur_state in ts_df.state.unique():
    ts_df[ts_df.state==cur_state].cumsum().plot(ax=ax, label = f'{cur_state}',xlim=(min(ts.index),datetime.datetime.now()))
ax.legend(ts_df.state.unique())

In [ ]:
fig, ax = plt.subplots(figsize=(30,15))
ax.set_yscale('log')
#ax.set_xticks(list(np.arange(min(ts.index.year),max(ts.index.year),1)),labels=np.arange(min(ts.index.year),max(ts.index.year),1))
for cur_state in ts_df.state.unique():
    ts_df[ts_df.state==cur_state].cumsum().plot(ax=ax, label = f'{cur_state}',xlim=(min(ts.index),datetime.datetime.now()),grid=True)
ax.legend(ts_df.state.unique())

## Primary takeaways/insights from time series analysis:

We can see that from the linear y axis line graph that there is a general trend of exponential increase across the board in alternative fuel stations for every state, with california leading the pack. However, by plotting the y-axis with a logarithmic scale, we can see that there are some periods of extreme growth across many states, creating a vertical jump in the number of stations. These time periods slightly vary among state, but there are a few periods that are basically unanimous, such as late 1999-2001, 2005-2006, 2012-2013, and 2021-2022. My current hypothesis is that:
1. these are related to national government policies, such as national subsidies for clean energy infrastructure
2. these are related to more widespread low emmision vehicle adoption
3. these are related to upgrades in the energy infrastructure across the nation
We could investigate these ideas by looking at government policy enactments surrounding these time frames, study overall clean energy vehicle sales, or investigating energy infrastructure upgrade reports.

In [ ]:
#import os
#os.system('jupyter nbconvert --to html EDA.ipynb')